<a href="https://colab.research.google.com/github/takatakashi/colab_web_demo/blob/master/ngrok_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2019-12-16 13:19:00--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.170.135.225, 52.204.223.154, 52.4.11.55, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.170.135.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  37.3MB/s    in 0.4s    

2019-12-16 13:19:01 (37.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]



In [0]:
!unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://91fa75a5.ngrok.io


In [0]:
import os

#Writing to html file on colab

html = """
<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8" />
</head>
<body>

Hello

</body>
</html>
"""

#Saving html file
if not os.path.isdir( "templates" ):
    os.makedirs( "templates" )
with open("templates/index.html", mode='w') as f:
    f.write(html)

In [0]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route("/")
def index():
    return render_template( "index.html" )

if __name__ == '__main__':
    app.run(port=6006)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6006/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Dec/2019 13:21:02] "GET / HTTP/1.1" 200 -


In [0]:
#@title
#Writing to html file on colab

html = """

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />
</head>
<body>

<form action="/post" method="post" enctype="multipart/form-data">
    <p><input type="file" name="uploadFile"/></p>
    <p><input type="submit" value="send"/></p>
</form>

{% if ulr_Image %}

    <p>Image</p>
    <p><img src="{{ ulr_Image }}"></p>
    <p>Result</p>
    <p>{{ result_class }}</p>
    <p>{{ result_score }}</p>

{% endif %}

</body>
</html>

"""

#Saving html file
if not os.path.isdir( "templates" ):
    os.makedirs( "templates" )
with open("templates/index.html", mode='w') as f:
    f.write(html)

#Making tmp folder to use temporarily stored
if not os.path.isdir( "tmp" ):
    os.makedirs( "tmp" )


In [0]:
!pip install scipy==1.1.0
!python -m pip install pillow

     |████████████████████████████████| 31.2MB 75kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.3
    Uninstalling scipy-1.3.3:
      Successfully uninstalled scipy-1.3.3


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


from flask import Flask, render_template, send_from_directory, request, redirect, url_for
from keras.models import load_model
import numpy as np
import os
from scipy.misc import imread, imsave, imresize
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
model = VGG16(weights='imagenet')
#Executing once to avoid errors
preds = model.predict(preprocess_input(np.zeros((1,224,224,3))))

def imageResize(img, h, w):
    ih, iw = img.shape[0], img.shape[1]
    if ih >= iw:
        dst = np.ones([ih, ih, 3], np.uint8) * 255
        dst[:, (ih - iw)//2:(ih - iw)//2 + iw] = img
    else:
        dst = np.ones([iw, iw, 3], np.uint8) * 255
        dst[(iw - ih)//2:(iw - ih)//2 + ih, :] = img

    dst = imresize(dst, (h, w))
    return dst        


app = Flask(__name__)

@app.route("/")
def index():
    return render_template( "index.html" )

@app.route('/post', methods=['POST'])
def post():
    try:
        uploadFile = request.files['uploadFile']
    except:
        return redirect(url_for('index'))

    uploadFile.save("tmp/" + uploadFile.filename)

    img = imread("tmp/" + uploadFile.filename)
    if img.shape[-1] == 4:
        img = img[:,:,0:3]
    img = imageResize(img, 224, 224)
    fileName = str(np.random.randint(1000)) + ".jpg"
    imsave("tmp/" + fileName, img)
    ulr_Image = "tmp/" + fileName

    img = np.expand_dims(img, axis=0)
    print(img.shape)
    preds = model.predict(preprocess_input(img))
    result = decode_predictions(preds, top=1)[0][0]
    result_class = "Class: " + result[1]
    result_score = "Score: " + str(result[2])

    return render_template( "index.html",
                                        ulr_Image = ulr_Image,
                                        result_class = result_class,
                                        result_score = result_score)

@app.route('/tmp/<filename>')
def uploaded_file(filename):
    return send_from_directory("./tmp", filename)

if __name__ == '__main__':
    app.run(port=6006)

https://0f3fcff3.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6006/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2019 07:53:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2019 07:53:54] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


(1, 224, 224, 3)
40960/35363 [==================================] - 0s 0us/step


127.0.0.1 - - [10/Dec/2019 07:54:05] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2019 07:54:05] "GET /tmp/607.jpg HTTP/1.1" 200 -


In [0]:
scipy.misc

NameError: ignored

In [0]:
from scipy.misc import imread, imsave, imresize

ImportError: ignored